In [1]:
# Step 0a: Load packages

# Import the xarray package
import xarray as xr
# Iport the numpy package
import numpy as np
# Import the math package
import math
# Import the module pyplot from the package matplotlib
import matplotlib.pyplot as plt
# Import the package os (operating system interface).
import os
# Import the cartopy package for plotting the coastline
import cartopy.crs as ccrs
# Import features of cartopy for plotting country borders
import cartopy.feature as cf
# Import datetime package for temporal interpolation
import pandas as pd
# Import widgets for interactive usage
import ipywidgets as ipyw
from IPython.display import display
# Import intake package
import intake
# Import easy gems with healpix
from easygems import healpix as egh
from easygems import remap as egr
# Import datetime
from datetime import datetime

In [2]:
# Global variables
pathEarthCare = '/work/mh0731/m301196/ecomip/ftp.eorc.jaxa.jp/eorc/CPR/1B/xCa/2024/08/01/'
engineType = 'netcdf4'
plotEarthCare = False
zoomFactor = 5

In [3]:
def readPath(fileName):

    filePath = pathEarthCare + fileName

    var = xr.open_dataset(filePath, engine=engineType)

    lon = np.array(var['lon']).astype(np.float64)
    lat = np.array(var['lat']).astype(np.float64)

    return lon, lat

In [4]:
def getTimeInterval(fileName):
    str_split = fileName.split('T')
    #year = str_split[0][-8:-4]
    year = "2020"
    month = str_split[0][-4:-2]
    day = str_split[0][-2:]
    start_time = str_split[1][:2] + ":" + str_split[1][2:4]
    end_time = str_split[2][:2] + ":" + str_split[2][2:4]

    timeInterval = []
    formatString = "%Y-%m-%dT%H:%M:%S"
    timeString = year + "-" + month + "-" + day + "T" + start_time + ":00"
    timeInterval.append(datetime.strptime(timeString, formatString))
    timeString = year + "-" + month + "-" + day + "T" + start_time + ":00"
    timeInterval.append(datetime.strptime(timeString, formatString))

    return timeInterval

In [5]:
def readVariable(variableName, fileName):

    filePath = pathEarthCare + fileName

    var = xr.open_dataset(filePath, engine=engineType)

    variable = np.array(var[variableName]).astype(np.float64)

    return variable

In [6]:
def plotEarthCarePath(lon, lat):
    # Set up a figure with Plate Carrée (rectangular) projection
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Add features: coastlines, borders, etc.
    ax.coastlines()
    ax.add_feature(cf.BORDERS, linestyle=':')
    ax.gridlines(draw_labels=True)

    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)

    # Add a title
    plt.title('EarthCare path')

    # Plot the points
    ax.plot(lon, lat, 'ro', markersize=5, transform=ccrs.PlateCarree())

    print(lon)
    print(lat)
    
    # Show the plot
    plt.show()

In [7]:
def plotVariable(variableName, var):
    # Add a title
    title = variableName
    plt.title(title)

    # Plot using indices as axes
    plt.imshow(var.T, origin='lower', cmap='viridis', aspect='auto')

    plt.gca().invert_yaxis()
        
    # Add colorbar
    plt.colorbar(label='Value')

    # Add axis labels and title
    plt.xlabel('Column Index')
    plt.ylabel('Row Index')
    plt.title('Plot curtain for ' + variableName)

    # Show the plot
    plt.show()

In [8]:
def plotVariableOverHeight(variableName, var, heights, lons):
    # Add a title
    title = variableName
    plt.title(title)

    # Plot the points
    height = heights.flatten()
    values = var.flatten()
    lon = np.zeros(len(values))

    for j in range(var.shape[0]):
        for k in range(var.shape[1]):
            lon[j * var.shape[1] + k] = lons[j]
    
    mask = np.isfinite(lon) & np.isfinite(height) & np.isfinite(values) & (height < 15000)

    lon_clean = lon[mask]
    height_clean = height[mask]
    values_clean = values[mask]
        
    # Plot using indices as axes
    plt.tricontourf(lon_clean, height_clean, values_clean, origin='lower', cmap='viridis')
        
    # Add colorbar
    plt.colorbar(label='Value')

    # Add axis labels and title
    plt.xlabel('longitude')
    plt.ylabel('height')
    plt.title('Plot curtain for ' + variableName)

    # Show the plot
    plt.show()

In [9]:
def plotEarthCareData(fileName, heights, lons, lats):

    outputB = ipyw.Output()
    #######################################################################################################################
    # Function to handle visualizations based on selections (is executed after pressing the button)
    def on_buttonB_click(b):

        ###################################################################################################################
        # Continue only if the some selection was made in the dropdown menu
        with outputB:

            ###########################################################################################################
            # Clear previous outputs
            outputB.clear_output()
            
            ###########################################################################################################
            # Retrieving the values from the widgets and store them in global variables
            varName = dropdownB.value

            ###########################################################################################################
            # Call the following function to continue
            variables = readVariable(varName, fileName)
            plotVariableOverHeight(varName, variables, heights, lons)

    filePath = pathEarthCare + fileName

    var = xr.open_dataset(filePath, engine=engineType)

    varNames = [str(v) for v in var if str(v) != 'lon' and str(v) != 'lat' and str(v) != 'height']
    
    dropdownB = ipyw.Dropdown(
        options=varNames,
        description='Variable selection:   ',
        style={'description_width': 'initial'},
        indent=False,
        layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
    )

    display(dropdownB)

    #######################################################################################################################
    # Button to trigger computation
    buttonB = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
    buttonB.on_click(on_buttonB_click)

    display(buttonB)

    display(outputB)

In [10]:
def plotSimulationData(fileName, heights, lons, lats, timeInter):

    outputB = ipyw.Output()
    #######################################################################################################################
    # Function to handle visualizations based on selections (is executed after pressing the button)
    def on_buttonB_click(b):

        ###################################################################################################################
        # Continue only if the some selection was made in the dropdown menu
        with outputB:

            ###########################################################################################################
            # Clear previous outputs
            outputB.clear_output()
            
            ###########################################################################################################
            # Retrieving the values from the widgets and store them in global variables
            location = dropdownB.value

            ###########################################################################################################
            # Call the following function to continue
            getSimulationModel(location, lons, lats, timeInter)

    availLocations = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")
    
    dropdownB = ipyw.Dropdown(
        options=availLocations,
        description='Location selection:   ',
        style={'description_width': 'initial'},
        indent=False,
        layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
    )

    display(dropdownB)

    #######################################################################################################################
    # Button to trigger computation
    buttonB = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
    buttonB.on_click(on_buttonB_click)

    display(buttonB)

    display(outputB)

In [11]:
def getSimulationModel(loc, lons, lats, timeInter):

    outputC = ipyw.Output()
    #######################################################################################################################
    # Function to handle visualizations based on selections (is executed after pressing the button)
    def on_buttonC_click(b):

        ###################################################################################################################
        # Continue only if the some selection was made in the dropdown menu
        with outputC:

            ###########################################################################################################
            # Clear previous outputs
            outputC.clear_output()
            
            ###########################################################################################################
            # Retrieving the values from the widgets and store them in global variables
            simModel = dropdownC.value
            zoomFactor = intSlider.value

            ###########################################################################################################
            # Call the following function to continue
            getVisualizationVariable(simModel, loc, lons, lats, zoomFactor, timeInter)
            

    availModels = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[loc]
    
    dropdownC = ipyw.Dropdown(
        options=availModels,
        description='Model selection:   ',
        style={'description_width': 'initial'},
        indent=False,
        layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
    )

    intSlider = ipyw.IntSlider(value=5, min=0, max=11, step=1, description='Zoom factor:', disabled=False,
                               continuous_update=False, orientation='horizontal', readout=True, readout_format='d',
                               indent=False, style={'description_width': 'initial'},
                               layout=ipyw.Layout(width='400px', margin='10px 20px 10px 0px')
    )

    hbox = ipyw.VBox([dropdownC, intSlider])
    display(hbox)

    #######################################################################################################################
    # Button to trigger computation
    buttonC = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
    buttonC.on_click(on_buttonC_click)

    display(buttonC)

    display(outputC)

In [16]:
def getVisualizationVariable(mod, loc, lons, lats, zoomFac, timeInter):

    outputD = ipyw.Output()
    #######################################################################################################################
    # Function to handle visualizations based on selections (is executed after pressing the button)
    def on_buttonD_click(b):

        ###################################################################################################################
        # Continue only if the some selection was made in the dropdown menu
        with outputD:

            ###########################################################################################################
            # Clear previous outputs
            outputD.clear_output()
            
            ###########################################################################################################
            # Retrieving the values from the widgets and store them in global variables
            varName = dropdownD.value

            ###########################################################################################################
            # Call the following function to continue
            interp_ds = interpolateData(ds, mod, loc, zoomFac, lons, lats, varName, timeInter)
            visualizeData(interp_ds)
        
    cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[loc]
    ds = cat[mod](zoom=zoomFac).to_dask()
    varNames = [str(v) for v in ds]
    
    dropdownD = ipyw.Dropdown(
        options=varNames,
        description='Model selection:   ',
        style={'description_width': 'initial'},
        indent=False,
        layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
    )

    display(dropdownD)

    #######################################################################################################################
    # Button to trigger computation
    buttonD = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
    buttonD.on_click(on_buttonD_click)

    display(buttonD)

    display(outputD)

In [17]:
def visualizeData(ds):

    fig, ax = plt.subplots()
    ds.isel(time=0).plot(x='track', y='pressure')

    plt.show()

In [18]:
def interpolateData(ds, mod, loc, zoomFac, lons, lats, selVar, selTime):

    # Find index of closest time before start_time
    start_idx = np.searchsorted(pd.to_datetime(ds.time.values), pd.to_datetime(selTime[0]), side='right') - 1
    # Find index of closest time after end_time
    end_idx = np.searchsorted(pd.to_datetime(ds.time.values), pd.to_datetime(selTime[1]), side='left')

    start_idx = max(start_idx, 0)
    end_idx = min(end_idx, len(pd.to_datetime(ds.time.values)) - 1)
    
    ds = (
        ds.rename_dims({"value": "cell"}).pipe(egh.attach_coords)
        if "value" in ds.dims
        else ds.pipe(egh.attach_coords)
    )
    
    ds_reduced = ds[selVar].isel(
        time=slice(start_idx, end_idx + 1)
    )
    
    ds_lon_deg = np.degrees(ds_reduced["lon"].values)
    ds_lat_deg = np.degrees(ds_reduced["lat"].values)
    weights = egr.compute_weights_delaunay(
        points=(ds_lon_deg, ds_lat_deg),
        xi=(lons, lats),
    )
    weights.to_netcdf('test.nc')

    ds_interpolated = xr.apply_ufunc(
        egr.apply_weights,
        ds_reduced,
        kwargs=weights,
        input_core_dims=[["cell"]],
        output_core_dims=[["track"]],
        output_dtypes=["f4"],
        vectorize=True,
        dask="parallelized",
        dask_gufunc_kwargs={
            "output_sizes": {"track": len(lons)},
        },
        keep_attrs=True,
    )

    return ds_interpolated

In [19]:
# Start the visualization script by filling out the basic setup
# Create an Output widget to display the results
outputA = ipyw.Output()

#######################################################################################################################
# Function to handle visualizations based on selections (is executed after pressing the button)
def on_buttonA_click(b):

    ###################################################################################################################
    # Continue only if the some selection was made in the dropdown menu
    with outputA:
        ###########################################################################################################
        # Retrieving the values from the widgets and store them in global variables
        pathFile = dropdown.value

        ###########################################################################################################
        # Clear previous outputs
        outputA.clear_output()

        ###########################################################################################################
        # Call the following function to continue with the setup
        lonCoord, latCoord = readPath(pathFile)
        timeInterval = getTimeInterval(pathFile)
        plotEarthCarePath(lonCoord, latCoord)
        heightCoord = readVariable('height', pathFile)
        if plotEarthCare:
            plotEarthCareData(pathFile, heightCoord, lonCoord, latCoord)
        else:
            plotSimulationData(pathFile, heightCoord, lonCoord, latCoord, timeInterval)

# List all files and folders
all_entries = os.listdir(pathEarthCare)

# Filter only files
files = [f for f in all_entries if os.path.isfile(os.path.join(pathEarthCare, f))]

dropdown = ipyw.Dropdown(
    options=files,
    description='Path selection:   ',
    style={'description_width': 'initial'},
    indent=False,
    layout=ipyw.Layout(width='35%', margin='30px 100px 30px 0px')
)

display(dropdown)

#######################################################################################################################
# Button to trigger computation
buttonA = ipyw.Button(description="Confirm selection", layout=ipyw.Layout(margin='30px 0px 30px 0px'))
buttonA.on_click(on_buttonA_click)

display(buttonA)

# Display the output widget to show the results
display(outputA)

Dropdown(description='Path selection:   ', layout=Layout(margin='30px 100px 30px 0px', width='35%'), options=(…

Button(description='Confirm selection', layout=Layout(margin='30px 0px 30px 0px'), style=ButtonStyle())

Output()